## 1. Importations

In [ ]:
import pandas as pd

from src.modeling import train_and_track_model_with_cv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
import mlflow

import warnings
import logging

# Masquer les warnings MLflow liés à l'environnement
warnings.filterwarnings("ignore", message=".*Failed to resolve installed pip version.*")
logging.getLogger("mlflow.utils.environment").setLevel(logging.ERROR)
mlflow.set_tracking_uri("http://localhost:5001")

## 2. Chargement des données

In [17]:
DATA_PATH = '../datas/02_preprocess/datas.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print(f"Données chargées. Forme: {df.shape}")
except FileNotFoundError:
    print(f"Erreur: Le fichier {DATA_PATH} n'a pas été trouvé. Vérifier le chemin.")

# Séparation des features (X) et de la cible (y)
if 'TARGET' in df.columns:
    X = df.drop('TARGET', axis=1)
    y = df['TARGET']
    print(f"X shape: {X.shape}, y shape: {y.shape}")
else:
    print("Erreur: La colonne 'TARGET' n'a pas été trouvée. Vérifier le nom de la colonne cible.")

Données chargées. Forme: (307507, 139)
X shape: (307507, 138), y shape: (307507,)


## 3. Simple séparation

In [18]:
# Division initiale pour avoir un ensemble de test final non touché par la CV
# Nous utiliserons X_train_full et y_train_full pour la validation croisée
X_train_full, X_test_final, y_train_full, y_test_final = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Taille de l'ensemble d'entraînement : {X_train_full.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test_final.shape[0]} échantillons")
print(f"Proportion de départs dans y_train : {y_train_full.mean():.2%}")
print(f"Proportion de départs dans y_test : {y_test_final.mean():.2%}")

Taille de l'ensemble d'entraînement : 246005 échantillons
Taille de l'ensemble de test : 61502 échantillons
Proportion de départs dans y_train : 8.07%
Proportion de départs dans y_test : 8.07%


## 4. Expérimentation par modèle

# Comparaison de modèles et validation croisée

Objectifs:
- Comparer plusieurs algorithmes de classification sur des données déséquilibrées.
- Utiliser une StratifiedKFold pour une évaluation robuste.
- Logger systématiquement les performances et artefacts dans MLflow.

Modèles testés:
- LogisticRegression (avec/without scaler, class_weight='balanced')
- RandomForest (class_weight='balanced')
- XGBoost / LightGBM (gestion native du déséquilibre)

Métriques suivies:
- ROC AUC, F1, Recall, Precision, PR AUC

Déséquilibre des classes:
- Class weights (selon le modèle)
- Option SMOTE (pipeline imblearn) quand pertinent

Sorties:
- Récapitulatif des performances (moyenne/écart CV)
- Artefacts: matrice de confusion, importance des features (arbres)
- Modèles enregistrés dans MLflow Model Registry

In [19]:
experiment_name = "Credit_Scoring_Comparison_CV"
mlflow.set_experiment(experiment_name)
print(f"MLflow Experiment set to: {experiment_name}")

2025/10/02 15:25:58 INFO mlflow.tracking.fluent: Experiment with name 'Credit_Scoring_Comparison_CV' does not exist. Creating a new experiment.


MLflow Experiment set to: Credit_Scoring_Comparison_CV


In [20]:
all_models_avg_metrics = {}

In [21]:
# --- Modèle 1: Régression Logistique ---
lr_params = {
    'solver': 'lbfgs',
    'max_iter': 2000,
    'C': 0.1,
    'class_weight': 'balanced'
}

lr_avg_metrics = train_and_track_model_with_cv(
    "LogisticRegression", LogisticRegression, lr_params, X_train_full, y_train_full, apply_scaler=True
)

all_models_avg_metrics["LogisticRegression_Scaled"] = lr_avg_metrics


--- Démarrage de l'expérimentation pour LogisticRegression ---
  LogisticRegression - Fold 1 metrics logged. ROC AUC: 0.7432
🏃 View run LogisticRegression_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/0b6aa1adee6d46eb9db6f9828e380371
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LogisticRegression - Fold 2 metrics logged. ROC AUC: 0.7411
🏃 View run LogisticRegression_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/c18b1ef2a6184707983894c3592b0cb9
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LogisticRegression - Fold 3 metrics logged. ROC AUC: 0.7408
🏃 View run LogisticRegression_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/83f99797c8b64949a7e5fc54e385fe9c
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LogisticRegression - Fold 4 metrics logged. ROC AUC: 0.7468
🏃 View run LogisticRegression_Fold_4 at: http://localho

Successfully registered model 'LogisticRegression_CreditScoring_CV_Scaler_True'.
2025/10/02 15:26:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression_CreditScoring_CV_Scaler_True, version 1


Pas de feature_importances_ disponible.
🏃 View run LogisticRegression_CV_Run_Scaler_True at: http://localhost:5000/#/experiments/672741076646558312/runs/3a5d54609a054114b952f7ed65d6a1b2
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour LogisticRegression terminée et loggée dans MLflow. ---


Created version '1' of model 'LogisticRegression_CreditScoring_CV_Scaler_True'.


In [22]:
# --- Modèle 1: Régression Logistique ---
lr_params = {
    'solver': 'lbfgs',
    'max_iter': 2000,
    'C': 0.1,
    'class_weight': 'balanced'
}
lr_avg_metrics = train_and_track_model_with_cv(
    "LogisticRegression",
    LogisticRegression,
    lr_params,
    X_train_full, y_train_full,
    apply_scaler=True,
    use_smote=True   # << actif pour LR
)
all_models_avg_metrics["LogisticRegression_Scaled_SMOTE"] = lr_avg_metrics


--- Démarrage de l'expérimentation pour LogisticRegression ---
  LogisticRegression - Fold 1 metrics logged. ROC AUC: 0.6830
🏃 View run LogisticRegression_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/3f260da838d44017b2d198ace31f971c
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LogisticRegression - Fold 2 metrics logged. ROC AUC: 0.6807
🏃 View run LogisticRegression_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/58354e6a05f34ff89cf53ff99d871c8a
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LogisticRegression - Fold 3 metrics logged. ROC AUC: 0.6808
🏃 View run LogisticRegression_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/d30f4fd809cc423bb76532fb9465d616
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LogisticRegression - Fold 4 metrics logged. ROC AUC: 0.6828
🏃 View run LogisticRegression_Fold_4 at: http://localho

Registered model 'LogisticRegression_CreditScoring_CV_Scaler_True' already exists. Creating a new version of this model...
2025/10/02 15:27:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LogisticRegression_CreditScoring_CV_Scaler_True, version 2


Pas de feature_importances_ disponible.
🏃 View run LogisticRegression_CV_Run_Scaler_True at: http://localhost:5000/#/experiments/672741076646558312/runs/cfc275bf8fff40e986a22f26693d00b4
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour LogisticRegression terminée et loggée dans MLflow. ---


Created version '2' of model 'LogisticRegression_CreditScoring_CV_Scaler_True'.


In [23]:
# --- Modèle 2: Random Forest (SANS StandardScaler - généralement pas nécessaire pour les arbres) ---
rf_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_leaf': 5,
    'class_weight': 'balanced',
    'n_jobs': -1
}
rf_avg_metrics = train_and_track_model_with_cv(
    "RandomForestClassifier", RandomForestClassifier, rf_params, X_train_full, y_train_full, apply_scaler=False
)
all_models_avg_metrics["RandomForestClassifier_NoScaler"] = rf_avg_metrics


--- Démarrage de l'expérimentation pour RandomForestClassifier ---
  RandomForestClassifier - Fold 1 metrics logged. ROC AUC: 0.7410
🏃 View run RandomForestClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/37ee7f6827dc4a63a3968da026bf8809
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  RandomForestClassifier - Fold 2 metrics logged. ROC AUC: 0.7392
🏃 View run RandomForestClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/90e6dbf1fe2a495382f8fe7c523d109a
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  RandomForestClassifier - Fold 3 metrics logged. ROC AUC: 0.7376
🏃 View run RandomForestClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/12d8940e24cd444ab157f661b255d74c
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  RandomForestClassifier - Fold 4 metrics logged. ROC AUC: 0.7424
🏃 View run RandomForest

Successfully registered model 'RandomForestClassifier_CreditScoring_CV_Scaler_False'.
2025/10/02 15:27:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier_CreditScoring_CV_Scaler_False, version 1
Created version '1' of model 'RandomForestClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run RandomForestClassifier_CV_Run_Scaler_False at: http://localhost:5000/#/experiments/672741076646558312/runs/958b2f39caa042beb46ef10e7fec568e
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour RandomForestClassifier terminée et loggée dans MLflow. ---


In [24]:
# --- Modèle 2: Random Forest ---
rf_params = {
    'n_estimators': 200,
    'max_depth': 10,
    'min_samples_leaf': 5,
    'n_jobs': -1
    # class_weight retiré si SMOTE activé
}
rf_avg_metrics = train_and_track_model_with_cv(
    "RandomForestClassifier",
    RandomForestClassifier,
    rf_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=True   # << actif pour RF
)
all_models_avg_metrics["RandomForestClassifier_SMOTE"] = rf_avg_metrics


--- Démarrage de l'expérimentation pour RandomForestClassifier ---
  RandomForestClassifier - Fold 1 metrics logged. ROC AUC: 0.6709
🏃 View run RandomForestClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/04fb1985b12b462686a8807268e0a0fb
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  RandomForestClassifier - Fold 2 metrics logged. ROC AUC: 0.6763
🏃 View run RandomForestClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/1ecfd6ecd5d248d6ba2044ed5d289904
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  RandomForestClassifier - Fold 3 metrics logged. ROC AUC: 0.6738
🏃 View run RandomForestClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/377bbbd726c24461885b17823e2e780e
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  RandomForestClassifier - Fold 4 metrics logged. ROC AUC: 0.6818
🏃 View run RandomForest

Registered model 'RandomForestClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/10/02 15:28:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier_CreditScoring_CV_Scaler_False, version 2
Created version '2' of model 'RandomForestClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run RandomForestClassifier_CV_Run_Scaler_False at: http://localhost:5000/#/experiments/672741076646558312/runs/972cb5f3f4b44198a9a0588e26d9711d
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour RandomForestClassifier terminée et loggée dans MLflow. ---


In [25]:
# --- Modèle 3: XGBoost ---
xgb_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'eval_metric': 'logloss',
    'scale_pos_weight': (y_train_full == 0).sum() / (y_train_full == 1).sum(),
    "tree_method": "hist",
    "n_jobs": -1
}
xgb_avg_metrics = train_and_track_model_with_cv(
    "XGBoostClassifier", XGBClassifier, xgb_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=False  # garder le ratio, pas de SMOTE ici
)
all_models_avg_metrics["XGBoostClassifier_NoSMOTE"] = xgb_avg_metrics


--- Démarrage de l'expérimentation pour XGBoostClassifier ---
  XGBoostClassifier - Fold 1 metrics logged. ROC AUC: 0.7626
🏃 View run XGBoostClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/10a66fedc4a449d3872f042352d6dfbe
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  XGBoostClassifier - Fold 2 metrics logged. ROC AUC: 0.7599
🏃 View run XGBoostClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/136a3c4eec664f8394fd44494d29fa5d
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  XGBoostClassifier - Fold 3 metrics logged. ROC AUC: 0.7596
🏃 View run XGBoostClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/39135bc0c1fb40899d6a8c12de82b4d8
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  XGBoostClassifier - Fold 4 metrics logged. ROC AUC: 0.7635
🏃 View run XGBoostClassifier_Fold_4 at: http://localhost:5000/#

Successfully registered model 'XGBoostClassifier_CreditScoring_CV_Scaler_False'.
2025/10/02 15:29:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostClassifier_CreditScoring_CV_Scaler_False, version 1
Created version '1' of model 'XGBoostClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run XGBoostClassifier_CV_Run_Scaler_False at: http://localhost:5000/#/experiments/672741076646558312/runs/3882c8b930b44306a34ae91b168dde23
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour XGBoostClassifier terminée et loggée dans MLflow. ---


In [26]:
# --- Modèle 3: XGBoost ---
xgb_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'eval_metric': 'logloss',
    'scale_pos_weight': (y_train_full == 0).sum() / (y_train_full == 1).sum(),
    "tree_method": "hist",
    "n_jobs": -1
}
xgb_avg_metrics = train_and_track_model_with_cv(
    "XGBoostClassifier", XGBClassifier, xgb_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=True  # garder le ratio, pas de SMOTE ici
)
all_models_avg_metrics["XGBoostClassifier_SMOTE"] = xgb_avg_metrics


--- Démarrage de l'expérimentation pour XGBoostClassifier ---
  XGBoostClassifier - Fold 1 metrics logged. ROC AUC: 0.7504
🏃 View run XGBoostClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/a691646d9b3e407fab178aefc502c23d
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  XGBoostClassifier - Fold 2 metrics logged. ROC AUC: 0.7472
🏃 View run XGBoostClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/9dbd2ca703c44123998ea4402b6870fd
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  XGBoostClassifier - Fold 3 metrics logged. ROC AUC: 0.7474
🏃 View run XGBoostClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/797321ed157841b29c697fe32869bbf5
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  XGBoostClassifier - Fold 4 metrics logged. ROC AUC: 0.7542
🏃 View run XGBoostClassifier_Fold_4 at: http://localhost:5000/#

Registered model 'XGBoostClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/10/02 15:29:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoostClassifier_CreditScoring_CV_Scaler_False, version 2
Created version '2' of model 'XGBoostClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run XGBoostClassifier_CV_Run_Scaler_False at: http://localhost:5000/#/experiments/672741076646558312/runs/b06ff4e8b4fa43cc9977ca436572f194
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour XGBoostClassifier terminée et loggée dans MLflow. ---


In [27]:
# --- Modèle 4: LightGBM ---
lgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'n_jobs': -1,
    'force_row_wise': True,
    'verbosity': -1
}
lgbm_avg_metrics = train_and_track_model_with_cv(
    "LightGBMClassifier", LGBMClassifier, lgbm_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=False  # garder is_unbalance
)
all_models_avg_metrics["LightGBMClassifier_NoSMOTE"] = lgbm_avg_metrics



--- Démarrage de l'expérimentation pour LightGBMClassifier ---
  LightGBMClassifier - Fold 1 metrics logged. ROC AUC: 0.7620
🏃 View run LightGBMClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/ed81fdac874a49468126ed4bbc3036da
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LightGBMClassifier - Fold 2 metrics logged. ROC AUC: 0.7612
🏃 View run LightGBMClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/fc24dcab55d14c988eac45181b54b0ca
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LightGBMClassifier - Fold 3 metrics logged. ROC AUC: 0.7600
🏃 View run LightGBMClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/b8844a3a55d34931bded2d234d34537b
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LightGBMClassifier - Fold 4 metrics logged. ROC AUC: 0.7652
🏃 View run LightGBMClassifier_Fold_4 at: http://localho

Successfully registered model 'LightGBMClassifier_CreditScoring_CV_Scaler_False'.
2025/10/02 15:30:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LightGBMClassifier_CreditScoring_CV_Scaler_False, version 1
Created version '1' of model 'LightGBMClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run LightGBMClassifier_CV_Run_Scaler_False at: http://localhost:5000/#/experiments/672741076646558312/runs/042ace403ed94bf0815a9fc70651aac2
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour LightGBMClassifier terminée et loggée dans MLflow. ---


In [28]:
# --- Modèle 4: LightGBM ---
lgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'n_jobs': -1,
    'force_row_wise': True,
    'verbosity': -1
}
lgbm_avg_metrics = train_and_track_model_with_cv(
    "LightGBMClassifier", LGBMClassifier, lgbm_params,
    X_train_full, y_train_full,
    apply_scaler=False,
    use_smote=True  # garder is_unbalance
)
all_models_avg_metrics["LightGBMClassifier_SMOTE"] = lgbm_avg_metrics


--- Démarrage de l'expérimentation pour LightGBMClassifier ---
  LightGBMClassifier - Fold 1 metrics logged. ROC AUC: 0.7616
🏃 View run LightGBMClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/912c9aa5a3e943009e56704b3bb8b406
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LightGBMClassifier - Fold 2 metrics logged. ROC AUC: 0.7566
🏃 View run LightGBMClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/98f4ebb537d44176a4b4e540adf3401f
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LightGBMClassifier - Fold 3 metrics logged. ROC AUC: 0.7576
🏃 View run LightGBMClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/d1714bc986484579add1de5b91baa816
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  LightGBMClassifier - Fold 4 metrics logged. ROC AUC: 0.7620
🏃 View run LightGBMClassifier_Fold_4 at: http://localho

Registered model 'LightGBMClassifier_CreditScoring_CV_Scaler_False' already exists. Creating a new version of this model...
2025/10/02 15:30:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LightGBMClassifier_CreditScoring_CV_Scaler_False, version 2
Created version '2' of model 'LightGBMClassifier_CreditScoring_CV_Scaler_False'.


Feature importance loggée: feature_importance.png
🏃 View run LightGBMClassifier_CV_Run_Scaler_False at: http://localhost:5000/#/experiments/672741076646558312/runs/48fb9dff89104a2db4561af5b6db6ab5
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour LightGBMClassifier terminée et loggée dans MLflow. ---


In [29]:
# --- Modèle 5: MLP (avec StandardScaler, option SMOTE) ---
mlp_params = {
    "hidden_layer_sizes": (128, 64),
    "activation": "relu",        # ReLU: rapide, évite saturations de tanh/sigmoid
    "alpha": 1e-4,
    "learning_rate": "adaptive",
    "max_iter": 100,
    "early_stopping": True
}
mlp_avg_metrics = train_and_track_model_with_cv(
    "MLPClassifier", MLPClassifier, mlp_params,
    X_train_full, y_train_full,
    apply_scaler=True,
    use_smote=True
)
all_models_avg_metrics["MLPClassifier_Scaled_SMOTE"] = mlp_avg_metrics


--- Démarrage de l'expérimentation pour MLPClassifier ---
  MLPClassifier - Fold 1 metrics logged. ROC AUC: 0.7255
🏃 View run MLPClassifier_Fold_1 at: http://localhost:5000/#/experiments/672741076646558312/runs/b89eafb11cce4df18433554e9c9c0e0d
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  MLPClassifier - Fold 2 metrics logged. ROC AUC: 0.7085
🏃 View run MLPClassifier_Fold_2 at: http://localhost:5000/#/experiments/672741076646558312/runs/4d222207e7dd4fedbace7cb35ddb7063
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  MLPClassifier - Fold 3 metrics logged. ROC AUC: 0.7208
🏃 View run MLPClassifier_Fold_3 at: http://localhost:5000/#/experiments/672741076646558312/runs/cfd2def31c0842f186d84bc543391d5f
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
  MLPClassifier - Fold 4 metrics logged. ROC AUC: 0.7127
🏃 View run MLPClassifier_Fold_4 at: http://localhost:5000/#/experiments/672741076646558312/runs

Successfully registered model 'MLPClassifier_CreditScoring_CV_Scaler_True'.
2025/10/02 15:34:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MLPClassifier_CreditScoring_CV_Scaler_True, version 1


Pas de feature_importances_ disponible.
🏃 View run MLPClassifier_CV_Run_Scaler_True at: http://localhost:5000/#/experiments/672741076646558312/runs/680678486ed1493eb547c2a8a5b35404
🧪 View experiment at: http://localhost:5000/#/experiments/672741076646558312
--- Expérimentation pour MLPClassifier terminée et loggée dans MLflow. ---


Created version '1' of model 'MLPClassifier_CreditScoring_CV_Scaler_True'.


In [30]:
# --- Comparaison finale des modèles ---
print("\n--- Récapitulatif des performances moyennes (ROC AUC) ---")
for model_name, metrics in all_models_avg_metrics.items():
    print(f"  {model_name}: Avg ROC AUC = {metrics['roc_auc']:.4f}")


--- Récapitulatif des performances moyennes (ROC AUC) ---
  LogisticRegression_Scaled: Avg ROC AUC = 0.7427
  LogisticRegression_Scaled_SMOTE: Avg ROC AUC = 0.6815
  RandomForestClassifier_NoScaler: Avg ROC AUC = 0.7394
  RandomForestClassifier_SMOTE: Avg ROC AUC = 0.6757
  XGBoostClassifier_NoSMOTE: Avg ROC AUC = 0.7611
  XGBoostClassifier_SMOTE: Avg ROC AUC = 0.7497
  LightGBMClassifier_NoSMOTE: Avg ROC AUC = 0.7621
  LightGBMClassifier_SMOTE: Avg ROC AUC = 0.7592
  MLPClassifier_Scaled_SMOTE: Avg ROC AUC = 0.7149
